# Counterfactual Risk Minimisation (CRM)

In [ ]:
!pip install recogym==0.1.2.1
!pip install git+git://github.com/criteo-research/bandit-reco.git#subdirectory=notebooks

In [ ]:
%matplotlib inline

import gym 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.autograd import Variable

from recogym.envs.session import OrganicSessions
from recogym import env_1_args, Configuration
from recogym import DefaultContext
from recogym.agents import Agent
from recogym.envs.observation import Observation
from recogym import verify_agents
from recogym.evaluate_agent import plot_verify_agents
from recogym.agents import FeatureProvider

from util import FullBatchLBFGS
from util import OrganicUserEventCounterAgent, organic_user_count_args

# Set style for pretty plots
plt.style.use('ggplot')

P = 10  # Number of Products
U = 2000 # Number of Users

# You can overwrite environment arguments here:
env_1_args['random_seed'] = 42
env_1_args['num_products'] = P
env_1_args['number_of_flips'] = P//2
env_1_args['sigma_mu_organic'] = 3
env_1_args['sigma_omega'] = 0.1

# Initialize the gym for the first time by calling .make() and .init_gym()
env = gym.make('reco-gym-v1')
env.init_gym(env_1_args)
env.reset()

## Data generation


We generate data with a popularity agent plus some randomness for exploration (`epsilon`).


In [ ]:
%%time
# Generate RecoGym logs for U users
logger = OrganicUserEventCounterAgent(Configuration({**organic_user_count_args,
                                                     'select_randomly': True,
                                                     'random_seed': 1209,
                                                     'epsilon': 0.001}))
reco_log = env.generate_logs(U, agent = logger)

Let's have a quick look at the logs

In [ ]:
reco_log.head(5)

We will use the popularity agent as the baseline. It basically predict the organic product that has been seen the most so far by the user. It is a simple but actually strong baseline.

In [ ]:
organic_counter_agent = OrganicUserEventCounterAgent(
    Configuration({**organic_user_count_args, 'select_randomly': False, 'epsilon': 0.0}))

In [ ]:
N_USERS_AB_TEST = 5000
result_AB = verify_agents(env, N_USERS_AB_TEST, {'User-pop': organic_counter_agent})
plot_verify_agents(result_AB)
None

## Contextual Bandits

### Data set transformation

In order to feed logs to a model, we need to transform them such that we have a dataset consisting in sequences of user context $x$, taken action $a$, reward $c$ and propensity scores $p$.

In [ ]:
class ProductCountFeatureProvider(FeatureProvider):
    """This feature provider creates a user state based on viewed product count.
    Namely, the feature vector of shape (n_products, ) contains for each product how many times the
    user has viewed them organically.
    """

    def __init__(self, config):
        super(ProductCountFeatureProvider, self).__init__(config)
        self.feature_data = np.zeros((self.config.num_products)).astype(int)

    def observe(self, observation):
        for session in observation.sessions():
            self.feature_data[int(session['v'])] += 1

    def features(self, observation):
        return self.feature_data.copy()

    def reset(self):
        self.feature_data[:] = 0


def build_rectangular_data(logs, feature_provider):
    """Create a rectangular feature set from the logged data.
    For each taken action, we compute the state in which the user was when the action was taken
    """
    user_states, actions, rewards, proba_actions = [], [], [], []
    
    current_user = None
    for _, row in logs.iterrows():
        if current_user != row['u']:
            # Use has changed: start a new session and reset user state
            current_user = row['u']
            sessions = OrganicSessions()
            feature_provider.reset()
        
        context = DefaultContext(row['u'], row['t'])
        
        if row['z'] == 'organic':
            sessions.next(context, row['v'])
            
        else:
            # For each bandit event, generate one observation for the user state, the taken action
            # the obtained reward and the used probabilities
            feature_provider.observe(Observation(context, sessions))
            user_states += [feature_provider.features(None)] 
            actions += [row['a']]
            rewards += [row['c']]
            proba_actions += [row['ps']] 
            
            # Start a new organic session
            sessions = OrganicSessions()
    
    return np.array(user_states), np.array(actions).astype(int), np.array(rewards), np.array(proba_actions)

Let's build and dissect this so-called rectangular dataset

In [ ]:
# You can now see data that will be provided to our agents based on logistic regressions
count_product_views_feature_provider = ProductCountFeatureProvider(config=Configuration(env_1_args))

In [ ]:
rectangular_logs = build_rectangular_data(reco_log, count_product_views_feature_provider)
user_states, actions, rewards, proba_actions = rectangular_logs

In [ ]:
preview_start, preview_size = 300, 3

print('user product views count at action time')
print(user_states[preview_start:preview_start + preview_size])
print('taken actions', actions[preview_start:preview_start + preview_size])
print('obtained rewards', rewards[preview_start:preview_start + preview_size])
print('probablities of the taken actions', proba_actions[preview_start:preview_start + preview_size])

### Vanilla contextual bandit

Classical value-based methods aim to learn the probability that a given action will lead to a positive reward, that is:

$$p(c = 1|\mathbf{x},a)$$

In what follows, we will implement a different approach: a policy learning method.
Policy-learning methods do not explicitly model the probability of a positive reward, but aim to directly model the action that should be taken, given a context:

$$p(a|\mathbf{x})$$

Classical contextual bandits achieve this by optimising the expectation of the reward under the new (learned) policy $\pi_\theta$, given a logged dataset under policy $\pi_0$.
For a given dataset $\mathcal{D}$ consisting of $N$ tuples $(\mathbf{x},a,p,c)$, the objective can be written as the following:

$$\theta^{*} = \text{argmax}_{\theta} \sum_{i=1}^{N}c_i\frac{\pi_\theta(a_i|\mathbf{x}_i)}{\pi_0(a_i|\mathbf{x}_i)}$$

This objective can be straightforwardly optimised using your favourite package that provides auto-differentiation functionality.
In our example, we will use PyTorch.

In [ ]:
class MultinomialLogisticRegressionModel(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        torch.nn.Module.__init__(self)
        # Generate weights - initialise randomly
        self.weight = torch.nn.Parameter(torch.Tensor(output_dim, input_dim))
        # torch.nn.init.kaiming_uniform_(self.weight, a = np.sqrt(5))
        torch.nn.init.zeros_(self.weight)

    def forward(self, x):
        inner_product = F.linear(x, self.weight)
        return F.softmax(inner_product, dim = 1)

class VanillaContextualBandit(Agent):
    def __init__(self, config, U = U, P = P, max_epoch=30):
        Agent.__init__(self, config)
        self.model = MultinomialLogisticRegressionModel(P, P)
        self.loss_history = []
        self.user_state = np.zeros(P)
        self.U = U
        self.P = P
        self.max_epoch = max_epoch

    def loss(self, X, a, proba_logged_actions, c):
        # Compute action predictions for clicks
        predicted_proba_for_all_actions = self.model(X)
        
        # Only keep probabilities for the actions that were taken
        predicted_proba = torch.gather(predicted_proba_for_all_actions, 1, a.unsqueeze(1)).reshape(-1)
        
        # expectation of the rewards under the new policy
        rewards = predicted_proba / proba_logged_actions

        # Since pytorch is meant to perform convex optimization, we rather
        # output a loss that we will want to minimize
        loss = - rewards
        return loss.mean()

    def train(self, rectangular_logs):
        """Train the contextual bandit based on an offline log such that it 
        learns to minimize its loss function
        """
        user_states, actions, rewards, proba_actions = rectangular_logs
        X = user_states
        a = actions
        p = proba_actions
        c = rewards
        
        # Put into PyTorch variables - drop unclicked samples
        X = Variable(torch.Tensor(X[c != 0]))
        a = Variable(torch.LongTensor(a[c != 0]))
        w = torch.Tensor(p[c != 0])
        
        def closure():
            # Reset gradients
            optimiser.zero_grad()

            return self.loss(X, a, w, c)
        
        # Set up optimiser
        optimiser = FullBatchLBFGS(self.model.parameters())

        # Initial loss
        self.loss_history.append(closure())
        max_epoch = self.max_epoch
        for epoch in range(max_epoch):
            # Optimisation step
            obj, _, _, _, _, _, _, _ = optimiser.step({'closure': closure,
                                                       'current_loss': self.loss_history[-1],
                                                       'max_ls': 20})
            self.loss_history.append(obj)
        
        return

    def plot_loss_history(self):
        ''' Plot the training loss over epochs '''
        _,_ = plt.subplots()
        plt.plot(range(len(self.loss_history)),self.loss_history)
        plt.xlabel('epoch')
        plt.ylabel('loss')
        plt.show()
        
    def observe(self, observation):
        ''' Observe new organic views and capture them in the user state '''
        for session in observation.sessions():
            self.user_state[int(session['v'])] += 1

    def act(self, observation, reward, done):
        ''' Pick an action, based on the current observation and the history '''
        # Observe
        self.observe(observation)

        # Act
        p_a = self.model(torch.Tensor([self.user_state])).detach().numpy().ravel()
        action = np.argmax(p_a)
        prob = np.zeros_like(p_a)
        prob[action] = 1.0

        return {
            **super().act(observation, reward, done),
            **{
                'a': action,
                'ps': 1.0,
                'ps-a': prob,
            },
        }

    def reset(self):
        ''' Reset the user state '''
        self.user_state = np.zeros(self.P)

In [ ]:
vanilla_cb = VanillaContextualBandit({}, max_epoch=200)
vanilla_cb.train(rectangular_logs)

In [ ]:
vanilla_cb.plot_loss_history()

In [ ]:
result_AB = pd.concat([result_AB, verify_agents(env, N_USERS_AB_TEST, {'Contextual Bandit': vanilla_cb})])
plot_verify_agents(result_AB)
None

It appears that some clicked events were actually sampled with a small probability ($\pi_0 \ll 1$). This might lead to overfitting for the contextual bandit and bad online performances.

In [ ]:
(reco_log[reco_log['c'] == 1]).sort_values('ps').head(5)

## Concave Contextual Bandit

In [ ]:
class ConcaveContextualBandit(VanillaContextualBandit):

    def loss(self, X, a, proba_logged_actions, c):
        # Compute action predictions for clicks
        predicted_proba_for_all_actions = self.model(X)
        
        # Only keep probabilities for the actions that were taken
        predicted_proba = torch.gather(predicted_proba_for_all_actions, 1, a.unsqueeze(1)).reshape(-1)
        
        # expectation of the reward under the new policy
        # code here
        # reward = 
        
        loss = -reward

        return loss.mean()

In [ ]:
log_cb = ConcaveContextualBandit({}, max_epoch=200)
log_cb.train(rectangular_logs)

In [ ]:
log_cb.plot_loss_history()

In [ ]:
result_AB = pd.concat([result_AB, verify_agents(env, N_USERS_AB_TEST, {'Concave contextual Bandit': log_cb})])
plot_verify_agents(result_AB)
None

## POEM

The classical IPS estimator has its disadvantages, however.
In its current form, the variance of the estimate can grow to be of significant size.
To mitigate this, Swaminathan and Joachims propose to include a sample variance penalisation term to the objective, effectively ensuring that the learned model does not stray too far from the logging policy.

This is the Counterfactual Risk Minimisation (CRM) objective, and the learning method optimising it directly is called POEM:
$$\theta^{*} = \text{argmax}_{\theta} \sum_{i=1}^{N}c_i\frac{\pi_\theta(a_i|\mathbf{x}_i)}{\pi_0(a_i|\mathbf{x}_i)} - \lambda \sqrt{\frac{\widehat{Var}_\theta}{N}} $$


We model $\pi_\theta$ as linear:

$$\pi_\theta(a|\mathbf{x}) = \text{softmax}(\mathbf{x}^{\intercal}\theta)_a$$


Swaminathan, Adith, and Thorsten Joachims. "Batch learning from logged bandit feedback through counterfactual risk minimization." Journal of Machine Learning Research 16.1 (2015): 1731-1755.

## POEM

In [ ]:
class PoemContextualBandit(VanillaContextualBandit):
    def __init__(self, config, U = U, P = P, max_epoch=30, variance_penalization_factor=0.):
        VanillaContextualBandit.__init__(self, config, U=U, P=P, max_epoch=max_epoch)
        self.variance_penalization_factor = variance_penalization_factor

    def loss(self, X, a, proba_logged_actions, c):
        # Compute action predictions for clicks
        predicted_proba_for_all_actions = self.model(X)
        
        # Only keep probabilities for the actions that were taken
        predicted_proba = torch.gather(predicted_proba_for_all_actions, 1, a.unsqueeze(1)).reshape(-1)
        
        # expectation of the loss under the new policy
        # code here
        # reward = 
        # Note: have a look at torch.sqrt and torch.var


In [ ]:
poem = PoemContextualBandit({}, variance_penalization_factor=0.4, max_epoch=100)
poem.train(rectangular_logs)

In [ ]:
poem.plot_loss_history()

In [ ]:
result_AB = pd.concat([result_AB, verify_agents(env, N_USERS_AB_TEST, {f'POEM': poem})])
plot_verify_agents(result_AB)
None